In [ ]:
import gensim
import pandas as pd
import numpy as np
import re

In [ ]:
embedding_path = 'D:/迅雷下载/wikipedia-pubmed-and-PMC-w2v.bin'
word2vec = gensim.models.KeyedVectors.load_word2vec_format(embedding_path,binary=True)

In [ ]:
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

In [ ]:
len(word2vec['heart'])

In [ ]:
file_name = 'C:/Users/wangkc/Desktop/胡喜风预测论文/text_flag_20200912.csv'
data = pd.read_csv(file_name)

In [ ]:
text = data[['hadm_id', 'text']].values
text[0, 0]
print(text)

In [ ]:
res_list = []
for hadm_id, item in text:
    new_text = re.sub('\[[^\[\]]*\]', ' ', item)
    new_text = re.sub('[#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”！[\\]^_`{|}~\s]+', " ", new_text)
    new_text = new_text.replace('\r\n', ' ').replace('\n', ' ').replace('\r', ' ').split(' ')
    new_text = list(filter(None, new_text))
    res_list.append([hadm_id, new_text])
len(res_list)

In [ ]:
vocab_data = []
for item in res_list: 
    res = np.zeros(200)
    num = 0
    none_num = 0
    for word in item[1]:
        if word in word2vec.wv.vocab:
            res += word2vec[word]
            num += 1
        elif word.lower() in word2vec.wv.vocab:
            print('lower: ', item[0], word)
            res += word2vec[word.lower()]
            num += 1
        elif word.upper() in word2vec.wv.vocab:
            print('upper: ', item[0], word)
            res += word2vec[word.upper()]
            num += 1
        else:
#             print(item[0], word)
            none_num += 1
    print('总数', item[0], num)
    print('None总数', item[0], none_num)
    vocab_data.append(np.append(item[0], res / num))
vocab_data = np.array(vocab_data)

In [ ]:
len(vocab_data)

In [ ]:
vocab_pd = pd.DataFrame(vocab_data)
vocab_pd

In [ ]:
vocab_pd['hadm_id'] = [int(item) for item in vocab_pd[0].values]
vocab_pd = vocab_pd.drop(labels=[0], axis=1)

In [ ]:
vocab_pd

In [ ]:
total_data = pd.merge(vocab_pd, data[['hadm_id', 'flag']], on='hadm_id', how='left')
total_data

In [ ]:
total_data = total_data.drop(labels=['hadm_id'], axis=1)

In [ ]:
total_data.shape

In [ ]:
total_data.corr()['flag']

In [ ]:
corr = total_data.corr()
expire_flag = corr['flag'].sort_values(ascending=False)
expire_flag[((expire_flag > 0.1) | (expire_flag < -0.1))]

In [ ]:
corr_data = total_data[expire_flag[(expire_flag > 0) | (expire_flag < 0)].index]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X, y = np.array(corr_data.drop(labels=['flag'], axis=1)), np.array([int(item) for item in corr_data['flag']])

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666,test_size = 0.33 )

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
standard_scaler = StandardScaler()
standard_scaler.fit(X_train)

In [ ]:
standard_scaler.mean_

In [ ]:
X_train_standard = standard_scaler.transform(X_train)
X_test_standard = standard_scaler.transform(X_test)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
lr = LogisticRegression(C=1000,random_state=666)
lr.fit(X_train_standard,y_train)
lr.score(X_test_standard,y_test) # 输出一个正确率

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
clf = SVC()
clf.fit(X_train_standard,y_train)
predictions = clf.predict(X_test_standard)
print("SVM")
print(classification_report(y_test,predictions))
print("AC",accuracy_score(y_test,predictions))